# General Utilities for Parts of the Langchain environment  

 > A library permits translation of complex boolean AND/OR queries between online APIs. 

In [ ]:
#| default_exp utils.langchain_utils
from nbdev import *

In [ ]:
#| export
import asyncio

import json
from typing import Dict, List, Optional, Sequence, Union

from langchain.callbacks.base import BaseCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models.ollama import ChatOllama
from langchain.llms import Ollama
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import AgentAction, AgentFinish, HumanMessage, AIMessage
from langchain.schema import BaseMessage, BaseOutputParser, OutputParserException
from langchain.schema import LLMResult
from langchain.utils.input import print_text

import os
import re
import sys
from typing import (
    Any,
    Dict,
    List,
    Union,
    Optional
)

import json
import re
from typing import Union

from langchain.agents.agent import AgentOutputParser
from langchain.agents.chat.prompt import FORMAT_INSTRUCTIONS
from langchain.schema import AgentAction, AgentFinish, OutputParserException

In [3]:
#| export

class suppress_stdout_stderr(object):
    def __enter__(self):
        self.outnull_file = open(os.devnull, 'w')
        self.errnull_file = open(os.devnull, 'w')

        self.old_stdout_fileno_undup    = sys.stdout.fileno()
        self.old_stderr_fileno_undup    = sys.stderr.fileno()

        self.old_stdout_fileno = os.dup ( sys.stdout.fileno() )
        self.old_stderr_fileno = os.dup ( sys.stderr.fileno() )

        self.old_stdout = sys.stdout
        self.old_stderr = sys.stderr

        os.dup2 ( self.outnull_file.fileno(), self.old_stdout_fileno_undup )
        os.dup2 ( self.errnull_file.fileno(), self.old_stderr_fileno_undup )

        sys.stdout = self.outnull_file        
        sys.stderr = self.errnull_file
        return self

    def __exit__(self, *_):        
        sys.stdout = self.old_stdout
        sys.stderr = self.old_stderr

        os.dup2 ( self.old_stdout_fileno, self.old_stdout_fileno_undup )
        os.dup2 ( self.old_stderr_fileno, self.old_stderr_fileno_undup )

        os.close ( self.old_stdout_fileno )
        os.close ( self.old_stderr_fileno )

        self.outnull_file.close()
        self.errnull_file.close()

In [ ]:
#| export

class OllamaRunner:
    '''Class to run Ollama in a subprocess and to  
     run LLMs or chains locally with a timeout to 
     prevent long-running processes from hanging 
     the server.'''
    proc = None
        
    def __init__(self, model):
        self.llm = Ollama(model=model)

    async def _start_server(self):
        if self.proc is not None:
            self._terminate_server()
        self.proc = await asyncio.create_subprocess_shell(
            'ollama serve',
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE)

    def _terminate_server(self):
        self.proc.terminate()
        self.proc = None

    def _callback(self, fut: asyncio.Future):
        if fut.cancelled() or not fut.done():
            print("Timed out! - Terminating server")
            fut.cancel()
            
    async def run_llm(self, prompt, timeout=300):
        # if server is not running, start it
        if self.proc is None:
            await self._start_server()
        # create task
        task = asyncio.create_task(self.llm.agenerate([prompt]))
        task.add_done_callback(self._callback)
        # try to await the task
        try:
            r = await asyncio.wait_for(task, timeout=timeout)
        except asyncio.TimeoutError as ex:
            print(ex)
        if r is not None:
            return '\n'.join([t[0].text for t in r.generations])
        else:
            return ''
        
    async def run_chain(self, chain, input, timeout=300):
        '''Incorporate the llm into a chain and run it.'''
        # if server is not running, start it
        if self.proc is None:
            await self._start_server()
        # create task
        task = asyncio.create_task(chain.ainvoke(input))
        task.add_done_callback(self._callback)
        # try to await the task
        try:
            r = await asyncio.wait_for(task, timeout=timeout)
        except asyncio.TimeoutError as ex:
            print(ex)
        return r

In [ ]:
#| export

# Hack from https://github.com/langchain-ai/langchain/issues/9917#issuecomment-1775620332
# to get the chat prompt to submit correctly to the Llama2 model

from langchain.schema import BaseMessage, HumanMessage, AIMessage, SystemMessage, FunctionMessage, ChatMessage

def get_buffer_string_for_llama(
    messages: Sequence[BaseMessage], human_prefix: str = "Human", ai_prefix: str = "AI"
) -> str:
    """Convert sequence of Messages to strings and concatenate them into one string.

    Args:
        messages: Messages to be converted to strings.
        human_prefix: The prefix to prepend to contents of HumanMessages.
        ai_prefix: THe prefix to prepend to contents of AIMessages.

    Returns:
        A single string concatenation of all input messages, 
        formatted for Llama Chat Models with <<SYS>>, <</SYS>>, [INST], and [/INST] tags.

    Example:
        .. code-block:: python

            from langchain.schema import AIMessage, HumanMessage

            messages = [
                HumanMessage(content="Hi, how are you?"),
                AIMessage(content="Good, how are you?"),
            ]
            get_buffer_string(messages)
            "<s><<SYS>>You are a helpful bot<</SYS>>
            [INST] Hi, how are you?[/INST]
            Good, how are you?"</s>
            <s>[INST]I'm fine, can you tell me who wrote 'Arabian Nights?'[/INST]
            Antoine Galland has been for three centuries, credited as being the author of the Arabian Nights.</s>
            ...
    """
    string_messages = []
    first_block = True 
    for m in messages:
        if isinstance(m, HumanMessage):
            if first_block:
                b = '[INST]' # assume the first message in the chain includes a system prompt and the remainders do not. 
                first_block = False 
            else:
                b = '<s>[INST]'
            e = '[/INST]'
        elif isinstance(m, AIMessage):
            b = ''
            e = '</s>' # assume the AI message is the last message of each block
        elif isinstance(m, SystemMessage):
            b = "<s><<SYS>>" # assume the systems message is the first message
            e = "<</SYS>>"
        elif isinstance(m, FunctionMessage):
            b = ''
            e = ''
        elif isinstance(m, ChatMessage):
            b = ''
            e = ''
        else:
            raise ValueError(f"Got unsupported message type: {m}")
        message = f"{b}{m.content}{e}"
        if isinstance(m, AIMessage) and "function_call" in m.additional_kwargs:
            message += f"{m.additional_kwargs['function_call']}"
        string_messages.append(message)

    return "\n".join(string_messages)

def ChatPromptValue_to_string(self) -> str:
    """Return prompt as string."""
    return get_buffer_string_for_llama(self.messages)


Put this code in your notebook to get the chat prompt to submit correctly to the Llama2 model
from langchain.prompts.chat import ChatPromptValue
```
ChatPromptValue.to_string = ChatPromptValue_to_string
```